#### Load custom .jsonl collection of city information extraction prompts and responses

In [1]:
import json
file_path = 'cities_qa.jsonl'

data = []
with open(file_path, 'r') as file:
    for line in file:
        json_line = json.loads(line.strip())
        data.append(json_line)

print(f"Number of QA pairs: {len(data)}")
print(data[0])

Number of QA pairs: 360
{'prompt': 'What are some popular outdoor activities to do in Montego Bay?', 'response': "Popular outdoor activities in Montego Bay include snorkeling and diving at the Montego Bay Marine Park, relaxing on the famous Doctor's Cave Beach, exploring the Green Grotto Caves, rafting on the Martha Brae River, and playing golf at world-class courses like the White Witch Golf Course and Cinnamon Hill Golf Course."}


#### Transform city prompt/response list of dictionary pairs into a list of dicts, which is suitable for Huggingface Dataset.from_dict()

In [2]:
transformed_data = {}
for key in data[0].keys():  # initialize keys
    transformed_data[key] = []

for item in data:
    for key in item:
        transformed_data[key].append(item[key])

#### Create a Huggingface dataset from list of dicts

In [3]:
from datasets import Dataset
dataset = Dataset.from_dict(transformed_data)
print(transformed_data)

{'prompt': ['What are some popular outdoor activities to do in Montego Bay?', 'Can you tell me about the history of Montego Bay?', 'What kind of weather can I expect in Montego Bay during the summer?', 'What are some traditional dishes I should try in Montego Bay?', 'Are there any local markets in Montego Bay where I can buy souvenirs?', 'What is the currency used in Montego Bay, and where can I exchange my money?', "I'm interested in the music scene in Montego Bay. What can I expect?", 'What are the main religious practices in Montego Bay?', 'How safe is Montego Bay for tourists?', 'What languages are spoken in Montego Bay?', 'Can you recommend any cafes in Montego Bay where I can try local coffee?', 'What is the weather like in Montego Bay throughout the year?', 'What are some popular dishes I should try while visiting Pristina?', 'Can you tell me about the climate in Pristina?', "I'm interested in outdoor activities. What can I do in Pristina?", 'What is the currency used in Pristin

#### Load the base pre-trained model and tokenizer
- Unsloth has it's own from_pretrained method.
- "load_in_4bit" indicates that the model will be quantized with bitsandbytes NormalFloat4 data type. This is the standard data type for QLoRA fine-tuning

In [4]:
from unsloth import FastLanguageModel
import torch
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = LOAD_IN_4BIT # Use 4bit quantization to reduce memory usage. Can be False.
)

False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('/usr/lib/nvidia-cuda-toolkit/lib64')}
The following directories listed in your path were found to be non-existent: {PosixPath('/usr/lib/nvidia-cuda-toolkit')}
The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {PosixPath('/home/ubuntu/.conda/envs/genaiplayground/etc/xml/catalog file'), PosixPath('/etc/xml/catalog'), PosixPath('file')}
The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: 

/home/ubuntu/.conda/envs/genaiplayground/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:167: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/home/ubuntu/.conda/envs/genaiplayground/lib/python3.11/site-packages/bitsandbytes/cuda_setup/main.py:167: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/ubuntu/.conda/envs/genaiplayground/lib/libcudart.so'), PosixPath('/home/ubuntu/.conda/envs/genaiplayground/lib/libcudart.so.11.0')}.. We select the PyTorch default libcudart.so, which is {torch.version.cuda},but this might missmatch with the CUDA version that is needed for bitsandbytes.To override this behavior set the BNB_CUDA_VERSION=<version string, e.g. 122> environmental variableFor example, if you want to use the CUDA version 122BNB_CUDA_VERSION=122 python ...OR set the environmental variable in your .bashrc: export BNB_CUDA_VERSION=122In the case of a manu

RuntimeError: 
        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

#### Transform our prompt/response dataset into Alpaca prompt template format 

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Will add EOS_TOKEN, otherwise your generation will go on forever!
def format_prompts(samples):
    instructions = samples["prompt"]
    responses = samples["response"]
    texts = []
    for prompt, response in zip(instructions, responses):
        text = alpaca_prompt.format(prompt, "", response) + EOS_TOKEN # Input is empty as no completion history
        texts.append(text)
    return { "text" : texts, }
pass

dataset_in_prompt_format = dataset.map(format_prompts, batched = True,)
print(dataset_in_prompt_format[0]['text'])


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What are some popular outdoor activities to do in Montego Bay?

### Input:


### Response:
Popular outdoor activities in Montego Bay include snorkeling and diving at the Montego Bay Marine Park, relaxing on the famous Doctor's Cave Beach, exploring the Green Grotto Caves, rafting on the Martha Brae River, and playing golf at world-class courses like the White Witch Golf Course and Cinnamon Hill Golf Course.</s>


#### Split the dataset into training and testing sets
Note by default train_test_split shuffles the data when splitting for random split.

In [6]:
train_test_split = dataset_in_prompt_format.train_test_split(test_size=0.1)  # 10% for testing
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']
print(f"Records in train set: {len(train_dataset)}")
print(f"Records in test set: {len(test_dataset)}")

Records in train set: 324
Records in test set: 36


#### Do model patching and add fast LoRA weights
r and lora_aplha are the most important parameters in LoRA configuration.

** r is the rank of the LoRA matrices:
- A higher r-value means more trainable parameters, allowing for more expressivity. But, on the negative side, there is a compute tradeoff, and may also lead to overfitting.
- A lower r-value means less trainable parameters, it can reduce overfitting at the cost of expressiveness.


** lora_aplha is a scaling factor for LoRA weights:
- Higher alpha will put more emphasis on LoRA weights.
- Lower alpha will put reduced emphasis on LoRA weights, hence model will be more dependent on its original weights.


** Important tips:
- Golden rule: lora_aplha = 2*r, i.e., if r=128 and lora_aplha should be 256
- Both r and lora_aplha should be in 2**x value, a good range for selection will be [8, 16, 32, 64, 128, 256, 512]
- If your fine-tuning data is very different from the pre-training data of your model, I recommend selecting r and lora_aplha from the higher values from the above range and vice versa.

In [7]:

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = MAX_SEQ_LENGTH,
)

Unsloth 2024.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


#### Define training arguments
Train for MAX_STEPS with a total batch size of 24 (per_device_train_batch_size*gradient_accumulation_steps)

In [8]:
MAX_STEPS = 120*2
from transformers import TrainingArguments
training_arguments = TrainingArguments(
        output_dir="./results",
        #evaluation_strategy="steps",
        #do_eval=True,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=6,
        #per_device_eval_batch_size=4,
        log_level="debug",
        save_steps=10,
        logging_steps=1, 
        learning_rate=2e-4,
        #eval_steps=2,
        optim='adamw_8bit',
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        weight_decay=0.1,
        max_steps=MAX_STEPS,
        warmup_ratio=0.01,
        lr_scheduler_type="linear",
)

#### Construct the model trainer
- Will train the model with TRL (Transformer Reinforcement Learning), with the SFT (Supervised Fine Tuning) trainer
- Use the text column of the dataset for training

In [9]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    train_dataset = train_dataset,
    #eval_dataset = test_dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    tokenizer = tokenizer,
    args = training_arguments,
)


Map:   0%|          | 0/324 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


#### Show current memory stats

In [10]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3090. Max memory = 23.691 GB.
4.625 GB of memory reserved.


#### Train the model

In [11]:
trainer_stats = trainer.train()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 324
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 6
  Total optimization steps = 12
  Number of trainable parameters = 41,943,040
Unsloth: `use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`


Step,Training Loss
1,1.628100
2,1.603100
3,1.169200
4,1.028600
5,0.925700
6,0.917500
7,0.881200
8,0.825300
9,0.795100
10,0.777200


Checkpoint destination directory ./results/checkpoint-10 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Saving model checkpoint to ./results/checkpoint-10
tokenizer config file saved in ./results/checkpoint-10/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-10/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




#### Show final memory and time stats

In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

61.266 seconds used for training.
1.02 minutes used for training.
Peak reserved memory = 5.805 GB.
Peak reserved memory for training = 1.18 GB.
Peak reserved memory % of max memory = 24.503 %.
Peak reserved memory for training % of max memory = 4.981 %.


#### Inference
Infer from the model using the earlier defined Alpca prompt format, leaving response blank

In [13]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence for the next 10 numbers.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # response, leaving blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence for the next 10 numbers.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\nThe next 10 numbers in the Fibonacci sequence are: 13, 21, 34, 55, 89, 144, 233, 377, 610, 987.</s>']

#### Inference with text streamer


In [14]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence for the next 10 numbers.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # response, leaving blank for generation!
    )
]*1, return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence for the next 10 numbers.

### Input:
1, 1, 2, 3, 5, 8

### Response:
The next 10 numbers in the Fibonacci sequence are: 13, 21, 34, 55, 89, 144, 233, 377, 610, 987.</s>


#### Saving only the LoRA adapters and NOT the full model
Using Huggingface's push_to_hub for an online save or save_pretrained for a local save.

In [ ]:
if False: model.save_pretrained("mistral_lora_model") # local saving
if False: model.push_to_hub("your_name/lora_model", token = "...") # Huggingface hub Online saving

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
HF_TOKEN = os.getenv('HF_TOKEN')

#### Example saving as transformers architecture for VLLM
Note unsloth push_to_hub_merged is same as HF push_to_hub, with additional perf features

Be aware if I save here, then it merges into n-bit then clears the LoRAs, so will get a NoneType error if later saving the save_pretrained_gguf format

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

#### GGUF / llama.cpp Conversion
Unsloth provides native GGUF/llama.cpp save. Clones llama.cpp and default save it to q8_0. Other quants include q4_k_m. Use save_pretrained_gguf for local saving and push_to_hub_gguf for uploading to HF.

In [ ]:
### GGUF / llama.cpp Conversion
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model_q4_k_m_gguf", tokenizer, quantization_method = "q4_k_m")
if True: model.push_to_hub_gguf("CorticalStack/travel-mistral-7B-GGUF", tokenizer, quantization_method = "q4_k_m", token = HF_TOKEN)